In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
os.environ["OMP_NUM_THREADS"] = "12"

import torch

In [2]:

import time
import yaml
import pandas as pd
from rdkit import Chem

In [ ]:
from rassp.msutil.masscompute import FragmentFormulaPeakEnumerator

#valid_atoms = {1, 5, 6, 7, 8, 9, 14, 15, 16, 17, 35, 53}
valid_atoms = {1, 6, 7, 8, 9, 15, 16, 17}
num_peaks_per_formula = 12
#max_formulae = 100000
max_formulae = 4096

ffe = FragmentFormulaPeakEnumerator(sorted(valid_atoms), use_highres=True, max_peak_num=num_peaks_per_formula)

def goodmol(mol):
        if len(mol.GetAtoms()) > 48:
            return False
        
        atoms = { a.GetAtomicNum() for a in mol.GetAtoms() }
        if not atoms < valid_atoms:
            return False
            
        f,m = ffe.get_frag_formulae(mol)
        if len(f) > max_formulae:
            return False

        return True


In [ ]:
test = pd.read_parquet('test.pq')
test

In [ ]:
smalltest = test[[ goodmol(Chem.AddHs(Chem.Mol(m))) for m in test.rdmol ]]
smalltest

In [ ]:
smalltest.to_parquet('smalltest.pq')

In [ ]:
train = pd.read_parquet('train.pq')
train

In [ ]:
smalltrain = train[[ goodmol(Chem.AddHs(Chem.Mol(m))) for m in train.rdmol ]]
smalltrain

In [ ]:
smalltrain.to_parquet('smalltrain.pq')

In [ ]:
len(train) + len(test)

In [ ]:
pd.read_parquet('smalltest.pq')

In [3]:
import sys
sys.path.append('./rassp')
from forward_train import train as rassp_train


In [4]:
with open('rassp/expconfig/demo.yaml') as cf:
    exp_config = yaml.load(cf,Loader=yaml.FullLoader)


In [12]:
exp_config['max_epochs'] = 3000
exp_config['epoch_size'] = 4096
exp_config['seed'] = int(time.time())
exp_config['batch_size'] = 8
exp_config['checkpoint_every_n_epochs'] = 1
exp_config['attempt_load_recent_checkpoint'] = True
exp_config['validate_every'] = 10
exp_config['cluster_config']['data_dir'] = '.'
exp_config['DATALOADER_NUM_WORKERS'] = 2
exp_config['exp_data']['data'][0]['db_filename'] = 'smalltrain.pq'
exp_config['exp_data']['data'][1]['db_filename'] = 'smalltest.pq'

with open('exp.yml','w') as y:
    yaml.dump(exp_config,y)


In [ ]:
%%capture
rassp_train('small',exp_config,exp_config_filename='exp.yml',USE_CUDA=True)